In [1]:
from pybtex.database import parse_file
import json
from pprint import pprint
import os

In [2]:
# changes names format to a single string (ex: "Robert Downey Jr.")

def name_filtering(data,test):
    for j in test :
        dataset = []
        if 'author' in test[j].persons:
            str = 'author'
        elif 'editor' in test[j].persons: 
            str = 'editor'
        else: 
            continue
        for i in test[j].persons[str]:
            first = ""
            middle = ""
            last = ""
            line = ""
            all = ""
            if i.first_names:
                all += i.first_names[0]
            if i.middle_names:
                all += " " + i.middle_names[0]
            if i.last_names:
                all += " " + i.last_names[0]
            if i.lineage_names:
                all += " " + i.lineage_names[0]
            dataset.append(all)
        data[j] = {'names': dataset, 'year': test[j].fields['year'], 'title': test[j].fields['title']}
    return data

In [64]:
# creates links and nodes 

def setup(data,ref_list,to_import):
    jfile = {}
    jfile["nodes"] = []
    jfile["links"] = []
    for article in data :
        to_do = False
        for k in data[article]['names']:
            if k in ref_list:
                to_do = True
                break
        if to_do:
            for i in data[article]['names']:
                node = {}
                node["id"] = i
                node["lastColaboration"] = "0"
                if i in ref_list:
                    node["group"] = find_team(i,to_import)
                else:
                    node["group"] = 0
                new_node = True
                for _node in jfile["nodes"]:
                    if _node["id"] == i:
                        new_node = False
                if new_node:
                    jfile["nodes"].append(node)
                for j in data[article]['names'] :
                    if (i != j) :
                        new = True
                        for k in jfile["links"]:
                            if (k["source"] == j) & (k["target"] == i) :
                                new = False
                                break
                            elif (k["source"] == i) & (k["target"] == j) :
                                if data[article]['title'] not in k["titles"]:
                                    k['titles'].append(data[article]['title'])
                                    k["nbLinks"] += 1
                                
                                link["lastColaboration"] = max(link["lastColaboration"],data[article]['year'])
                                new = False
                                break
                        if new:
                            link = {}
                            link["source"] = i
                            link["target"] = j
                            link["nbLinks"] = 1
                            link["titles"] = [data[article]['title']]
                            link["lastColaboration"] = data[article]['year']
                            jfile["links"].append(link)
                                                        
                for j in range(len(jfile["nodes"])):
                    if jfile["nodes"][j]["id"] == i:
                        jfile["nodes"][j]["lastColaboration"] = max(jfile["nodes"][j]["lastColaboration"],data[article]['year'])
                        break
                    
    return jfile


In [35]:
def clean_data(data):
    tab = []
    tab.append(['{\\\'{e}}','é'])
    tab.append(['{\\\'{E}}','É'])
    tab.append(['{-}','-'])
    tab.append(['{\\^{o}}','ô'])
    tab.append(['{\\"{e}}','ë'])
    tab.append(['{\\"{u}}','ü'])
    tab.append(['{\\`{e}}','è'])
    tab.append(['{\\`{o}}','ò'])
    tab.append(['{\\"{a}}','ä'])
    tab.append(['{\\ae}','æ'])
    tab.append(['{\\ss}','ß'])
    tab.append(['{\\"{o}}','ö'])
    tab.append(['{\\"{\\i}}','ï'])
    tab.append(['{\\\'{a}}','à'])
    tab.append(['{\\^{\\i}}','î'])
    tab.append(['{\\\'{\\i}}','í'])
    tab.append(['{\\~{n}}','ñ'])
    tab.append(['{\\c{c}}','ç'])
    tab.append(['{\oe}','œ'])
    
    
    for article in data :
        for pair in tab:
            for i in range(len(data[article]['names'])):
                data[article]['names'][i] = data[article]['names'][i].replace(pair[0],pair[1])
            data[article]['title'] = data[article]['title'].replace(pair[0],pair[1])
        

In [5]:
#save results in json file

def save(jfile):
    with open('test.json', 'w') as f:
        json.dump(jfile, f)

    pprint(jfile)

In [6]:
to_import = [['Romain','Vuillemot'],
             ['Audrey','Serna'],
             ['Christine','Michel'],
             ['Karim','Sehaba'],
             ['Élise','Lavoué'],
             ['Benoît','Encelle'],
             ['René','Chalon'],
             ['Jean-Charles','Marty'],
             ['Françoise','Sandoz-Guermond'],
             ['Aurélien','Tabard']]
ref_list = []
for i in to_import:
    ref_list += [i[0] + ' ' + i[1]]

In [7]:
def retrieve_files():
    to_import = []
    for x in os.walk(os.getcwd() + "\\bibtex_data"):
        if x[2]:
            team_name = x[0].replace(os.getcwd() + "\\bibtex_data\\", '')
            data = {'team': team_name, 'persons': []}
            for i in x[2]:
                i = i.replace('.bib','')
                data['persons'].append(i.split('_'))
            to_import.append(data)

    ref_list = []
    for i in range(len(to_import)):
        for j in to_import[i]['persons']:
            ref_list += [j[1] + ' ' + j[0]]
    
    return to_import,ref_list

In [8]:
def find_team(name,to_import):
    usable_name = list(reversed(name.split(" ")))
    for team in to_import:
        if usable_name in team["persons"]:
            return team["team"]
    return "unknown_team"

In [65]:
filtered_data = {}

to_import,ref_list = retrieve_files()

for team_names in to_import:
    for people in team_names['persons']:
        data = parse_file('bibtex_data/' + team_names['team'] + '/'+ people[0] + '_' + people[1] + '.bib').entries
        filtered_data = name_filtering(filtered_data,data)
        clean_data(filtered_data)
       
jfile = setup(filtered_data,ref_list,to_import)
save(jfile)

{'links': [{'lastColaboration': '2018',
            'nbLinks': 28,
            'source': 'Bertrand David',
            'target': 'René Chalon',
            'titles': ["Box/Lockers' contribution to Collaborative Economy in "
                       'the Smart City',
                       'A context-aware mobile system for work-based learning',
                       'A smart brain: an intelligent context inference engine '
                       'for context-aware middleware',
                       'Interaction on-the-go: a fine-grained exploration on '
                       'wearable {PROCAM} interfaces and gestures in mobile '
                       'situations',
                       'Collaborative Systems {\\&} Shared Economy '
                       '(Uberization): Principles {\\&} Case Study',
                       'Eight Years of R{\\&}D in the Freight {\\&} Passenger '
                       'Transportation Context in the Smart City Perspective: '
                       '{IC

           {'lastColaboration': '2014',
            'nbLinks': 1,
            'source': 'José Rouillard',
            'target': 'René Chalon',
            'titles': ['Rapid Prototyping for Mobile Serious Games']},
           {'lastColaboration': '2014',
            'nbLinks': 1,
            'source': 'Audrey Serna',
            'target': 'Bertrand David',
            'titles': ['Rapid Prototyping for Mobile Serious Games']},
           {'lastColaboration': '2014',
            'nbLinks': 1,
            'source': 'Audrey Serna',
            'target': 'René Chalon',
            'titles': ['Rapid Prototyping for Mobile Serious Games']},
           {'lastColaboration': '2014',
            'nbLinks': 2,
            'source': 'Chen Wang',
            'target': 'Bertrand David',
            'titles': ['A Smart City Case Study: Dynamic Management of Road '
                       'Lanes',
                       'Dynamic road lane management: {A} smart city '
                       'application']

            'source': 'Vineet Gandhi',
            'target': 'Florent Thiery',
            'titles': ['Capturing and Indexing Rehearsals: The Design and '
                       'Usage of a Digital Archive of Performing Arts']},
           {'lastColaboration': '2015',
            'nbLinks': 1,
            'source': 'Cyrille Migniot',
            'target': 'Florent Thiery',
            'titles': ['Capturing and Indexing Rehearsals: The Design and '
                       'Usage of a Digital Archive of Performing Arts']},
           {'lastColaboration': '2015',
            'nbLinks': 2,
            'source': 'Madjid Sadallah',
            'target': 'Benoît Encelle',
            'titles': ['Towards Reading Session-Based Indicators in '
                       'Educational Reading Analytics',
                       'A framework for usage-based document reengineering']},
           {'lastColaboration': '2015',
            'nbLinks': 2,
            'source': 'Madjid Sadallah',
            'ta

            'source': 'Hendrik Drachsler',
            'target': 'Mar Pérez-Sanagustín',
            'titles': ['Data Driven Approaches in Digital Education - 12th '
                       'European Conference on Technology Enhanced Learning, '
                       '{EC-TEL} 2017, Tallinn, Estonia, September 12-15, '
                       '2017, Proceedings']},
           {'lastColaboration': '2017',
            'nbLinks': 1,
            'source': 'Katrien Verbert',
            'target': 'Julien Broisin',
            'titles': ['Data Driven Approaches in Digital Education - 12th '
                       'European Conference on Technology Enhanced Learning, '
                       '{EC-TEL} 2017, Tallinn, Estonia, September 12-15, '
                       '2017, Proceedings']},
           {'lastColaboration': '2017',
            'nbLinks': 1,
            'source': 'Katrien Verbert',
            'target': 'Mar Pérez-Sanagustín',
            'titles': ['Data Driven Approaches in Digit

            'titles': ['{PLM} Serious Game Approach Available Both for Change '
                       'Management and Knowledge Assessment']},
           {'lastColaboration': '2014',
            'nbLinks': 1,
            'source': 'S. Elkadiri',
            'target': 'Aurélie Bissay',
            'titles': ['{PLM} Serious Game Approach Available Both for Change '
                       'Management and Knowledge Assessment']},
           {'lastColaboration': '2014',
            'nbLinks': 1,
            'source': 'S. Elkadiri',
            'target': 'Jean-Charles Marty',
            'titles': ['{PLM} Serious Game Approach Available Both for Change '
                       'Management and Knowledge Assessment']},
           {'lastColaboration': '2014',
            'nbLinks': 1,
            'source': 'Aurélie Bissay',
            'target': 'Jean-Charles Marty',
            'titles': ['{PLM} Serious Game Approach Available Both for Change '
                       'Management and Knowledge

...

            'source': 'Rim Bentebibel',
            'target': 'Amedeo Napoli',
            'titles': ['{TAAABLE:} Text Mining, Ontology Engineering, and '
                       'Hierarchical Classification for Textual Case-Based '
                       'Cooking']},
           {'lastColaboration': '2008',
            'nbLinks': 1,
            'source': 'Rim Bentebibel',
            'target': 'Yannick Toussaint',
            'titles': ['{TAAABLE:} Text Mining, Ontology Engineering, and '
                       'Hierarchical Classification for Textual Case-Based '
                       'Cooking']},
           {'lastColaboration': '2008',
            'nbLinks': 1,
            'source': 'Pierre-Antoine Champin',
            'target': 'Julien Cojan',
            'titles': ['{TAAABLE:} Text Mining, Ontology Engineering, and '
                       'Hierarchical Classification for Textual Case-Based '
                       'Cooking']},
           {'lastColaboration': '2008',
            '

                       'Thematic Question Generation over Knowledge Bases',
                       'Real-time, Scalable, Content-based Twitter Users '
                       'Recommendation',
                       'Upsortable: Programming TopK Queries Over Data Streams',
                       'Pi-CEP: Predictive Complex Event Processing Using '
                       'Range Queries over Historical Pattern Space',
                       'Docker2RDF: Lifting the Docker Registry Hub into {RDF}',
                       'Inferray: fast in-memory {RDF} inference',
                       'Real-time, scalable, content-based Twitter users '
                       'recommendation',
                       'Incremental and Directed Rule-Based Inference on '
                       '{RDFS}',
                       'Slider : un Raisonneur Incrémental Évolutif',
                       'Plongement de métrique pour le calcul de similarité '
                       "sémantique {\\`{a}} l'échelle",
     

           {'lastColaboration': '2017',
            'nbLinks': 2,
            'source': 'Xavier Dolques',
            'target': 'Laura Martin',
            'titles': ["Raisonner {\\`{a}} partir de cas d'allocation dans des "
                       'parcelles agricoles',
                       'A Reasoning Model Based on Perennial Crop Allocation '
                       'Cases and Rules']},
           {'lastColaboration': '2017',
            'nbLinks': 2,
            'source': 'Xavier Dolques',
            'target': 'Alain Mille',
            'titles': ["Raisonner {\\`{a}} partir de cas d'allocation dans des "
                       'parcelles agricoles',
                       'A Reasoning Model Based on Perennial Crop Allocation '
                       'Cases and Rules']},
           {'lastColaboration': '2017',
            'nbLinks': 2,
            'source': 'Xavier Dolques',
            'target': 'Marc Benoît',
            'titles': ["Raisonner {\\`{a}} partir de cas d'allocation 

            'source': 'Lionel Médini',
            'target': 'Nicolas Armando',
            'titles': ['DataConf: {A} Full Client-Side Web Mashup for '
                       'Scientific Conferences']},
           {'lastColaboration': '2013',
            'nbLinks': 2,
            'source': 'Florian B{\\^{a}}cle',
            'target': 'Benoît Durant Pastellière',
            'titles': ['DataConf: {A} Full Client-Side Web Mashup for '
                       'Scientific Conferences',
                       'DataConf and Its Linked Open Data Ecosystem: Produce, '
                       'Link and Consume Scientific Conference Metadata']},
           {'lastColaboration': '2013',
            'nbLinks': 2,
            'source': 'Florian B{\\^{a}}cle',
            'target': 'Fiona Le Peutrec',
            'titles': ['DataConf: {A} Full Client-Side Web Mashup for '
                       'Scientific Conferences',
                       'DataConf and Its Linked Open Data Ecosystem: Produce, '
  

            'lastColaboration': '2014'},
           {'group': 0, 'id': 'Laurianne Charrier', 'lastColaboration': '2019'},
           {'group': 0, 'id': 'Alisa Rieger', 'lastColaboration': '2019'},
           {'group': 0, 'id': 'Alexandre Galdeano', 'lastColaboration': '2019'},
           {'group': 0, 'id': 'Amélie Cordier', 'lastColaboration': '2019'},
           {'group': 'SMA', 'id': 'Mathieu Lefort', 'lastColaboration': '2019'},
           {'group': 0, 'id': 'A. Chekir', 'lastColaboration': '2017'},
           {'group': 0, 'id': 'Maxime Descoteaux', 'lastColaboration': '2017'},
           {'group': 0,
            'id': 'Marc-Alexandre Côté',
            'lastColaboration': '2017'},
           {'group': 0,
            'id': 'Eleftherios Garyfallidis',
            'lastColaboration': '2017'},
           {'group': 0,
            'id': 'Fatima Oulebsir-Boumghar',
            'lastColaboration': '2017'},
           {'group': 0, 'id': 'Victor Lequay', 'lastColaboration': '2017'},
        